In [1]:
from collections import defaultdict
from copy import copy
import numpy as np
import operator
import os
import pandas as pd
import pickle

import logging
logging.basicConfig(level=logging.DEBUG,
        format='%(levelname)-8s [%(lineno)d] %(message)s')

In [2]:
tensor_dir = '/mnt/permanent/home/makrai/project/verb-tensor/top_level/tensor'

# Task 1: One-mode similarity

## Data: [SimVerb](http://people.ds.cam.ac.uk/dsg40/simverb.html) (Gerz+ EMNLP 2016)

In [3]:
simverb = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/simverb-3500/SimVerb-3500.txt', sep='\t',
                      header=None, names=['verb1', 'verb2', 'pos', 'sim', 'rel'])

In [4]:
simverb.describe(percentiles=[])

,sim
count,3500.000000
mean,4.291554
std,2.652621
min,0.000000
50%,4.320000
max,9.960000


In [5]:
simverb.head()

,verb1,verb2,pos,sim,rel
0,take,remove,V,6.81,SYNONYMS
1,walk,trail,V,4.81,COHYPONYMS
2,feed,starve,V,1.49,ANTONYMS
3,shine,polish,V,7.80,SYNONYMS
4,calculate,add,V,5.98,HYPER/HYPONYMS


In [6]:
simverb.groupby('rel').sim.describe(percentiles=[]).sort_values('count', ascending=False)

,count,mean,std,min,50%,max
rel,,,,,,
NONE,2093.0,3.431276,2.342695,0.0,3.150,9.79
HYPER/HYPONYMS,800.0,6.012525,2.104537,0.5,6.310,9.96
SYNONYMS,306.0,6.789150,2.104490,0.5,7.140,9.96
COHYPONYMS,190.0,4.435526,2.381992,0.0,4.665,9.30
ANTONYMS,111.0,0.977748,1.074232,0.0,0.660,6.04


## Data: SimLex-999

In [7]:
simlex = pd.read_csv('/mnt/permanent/Language/English/Data/SimLex-999/SimLex-999.txt', sep='\t')

In [8]:
simlex.head()

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93


* conc(w1): The concreteness rating of word1 on a scale of 1-7. Taken from the University of South Florida Free Association Norms database. 

* conc(w2): The concreteness rating of word2 on a scale of 1-7. Taken from the University of South Florida Free Association Norms database. 

* concQ: The quartile the pair occupies based on the two concreteness ratings. Used for some analyses in the above paper. 

* Assoc(USF): The strength of free association from word1 to word2. Values are taken from the University of South Florida Free Association Dataset. 

* SimAssoc333: Binary indicator of whether the pair is one of the 333 most associated in the dataset (according to Assoc(USF)). This subset of SimLex999 is often the hardest for computational models to capture because the noise from high association can confound the similarity rating. See the paper for more details. 

* SD(SimLex): The standard deviation of annotator scores when rating this pair. Low values indicate good agreement between the 15+ annotators on the similarity value SimLex999. Higher scores indicate less certainty. 


In [9]:
simlex.describe(percentiles=[])

,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,4.561572,3.657087,3.568629,2.501502,0.751512,0.333333,1.274505
std,2.614663,1.131050,1.159572,1.118145,1.344569,0.471641,0.366278
min,0.230000,1.190000,1.190000,1.000000,0.000000,0.000000,0.340000
50%,4.670000,3.830000,3.660000,3.000000,0.250000,0.000000,1.310000
max,9.800000,5.000000,5.000000,4.000000,8.850000,1.000000,2.180000


In [10]:
simlex.groupby('POS').size()

POS
A    111
N    666
V    222
dtype: int64

## Testing the verb tensor

In [77]:
def test_sim(weight, cutoff=2, max_rank=256, verb=True, normalize=True, lmbda=False):
    stensor, index = pickle.load(open(
        os.path.join(tensor_dir, 'sparstensr_{}_{}.pkl').format(weight, cutoff),
        mode='rb'))
    logging.info(stensor.shape)
    target_df = copy(simverb if verb else simlex)
    target_cols = ['verb1', 'verb2'] if verb else ['word1', 'word2']
    relation = 'ROOT' if verb else 'nsubj'
    mode = 1 if verb else 0
    def tensor_based_sim(rank):
        oov = defaultdict(int)
        ktensor, fit, n_iterations, exectimes = pickle.load(open(
            os.path.join(tensor_dir, 'ktensor_{}_{}_{}.pkl').format(weight, cutoff, rank), mode='rb'))
        #modes = ['nsubj', 'ROOT', 'dobj']
        if lmbda:
            sq_lam = np.sqrt(np.apply_along_axis(np.linalg.norm, 0, ktensor.lmbda))
            ktensor.U[mode] *= sq_lam
        if normalize:
            ktensor.U[mode] /= np.apply_along_axis(np.linalg.norm, 1, ktensor.U[mode]).reshape((-1,1))
        def pointwise_prod_more(ser):
            try:
                # TODO add option for lmbda
                return ktensor.U[mode][index[relation][ser[0]]].dot(ktensor.U[mode][index[relation][ser[1]]])
            except KeyError as e:
                oov[e.args] += 1
                # TODO 0?
                return 0
        target_df['tensor_sim_{}'.format(rank)] = target_df[target_cols].apply(pointwise_prod_more, axis=1)
        if rank == max_rank:
            logging.debug(sorted(oov.items(), key=operator.itemgetter(1), reverse=True)[:5])
    for exp in range(4, int(np.log2(max_rank))+1):
        rank = 2**exp
        try:
            tensor_based_sim(rank=rank)
        except FileNotFoundError as e:
            logging.warning(e)
            target_df['tensor_sim_{}'.format(rank)] = 0
    sim_col = 'sim' if verb else 'SimLex999'
    print(target_df.corr(method='spearman').loc[sim_col][0 if verb else 7 :])

In [78]:
for weight in ['log_freq', 'pmi', 'iact_info', 'salience', 'iact_sali', 'log_dice', 'dice_sali']:
    print('\n{}'.format(weight))
    test_sim(weight)


log_freq


INFO     [5] (44277, 9292, 39360)
/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in true_divide
DEBUG    [30] [(('disintegrate',), 17), (('disown',), 14), (('perish',), 14), (('deteriorate',), 13), (('kneel',), 9)]


sim               1.000000
tensor_sim_16     0.057205
tensor_sim_32     0.026298
tensor_sim_64     0.067312
tensor_sim_128    0.034598
tensor_sim_256    0.025893
Name: sim, dtype: float64

pmi


INFO     [5] (44277, 9292, 39360)
DEBUG    [30] [(('disintegrate',), 17), (('disown',), 14), (('perish',), 14), (('deteriorate',), 13), (('kneel',), 9)]


sim               1.000000
tensor_sim_16     0.029379
tensor_sim_32     0.023028
tensor_sim_64     0.017924
tensor_sim_128    0.095454
tensor_sim_256    0.012126
Name: sim, dtype: float64

iact_info


...

sim               1.000000
tensor_sim_16     0.031748
tensor_sim_32    -0.002595
tensor_sim_64     0.038968
tensor_sim_128    0.055166
tensor_sim_256         NaN
Name: sim, dtype: float64

log_dice


INFO     [5] (44277, 9292, 39360)
WARNING  [36] [Errno 2] No such file or directory: '/mnt/permanent/home/makrai/project/verb-tensor/top_level/tensor/ktensor_log_dice_2_256.pkl'


sim               1.000000
tensor_sim_16    -0.001559
tensor_sim_32     0.031213
tensor_sim_64     0.107157
tensor_sim_128    0.091232
tensor_sim_256         NaN
Name: sim, dtype: float64

dice_sali


INFO     [5] (44277, 9292, 39360)
WARNING  [36] [Errno 2] No such file or directory: '/mnt/permanent/home/makrai/project/verb-tensor/top_level/tensor/ktensor_dice_sali_2_256.pkl'


sim               1.000000
tensor_sim_16     0.039431
tensor_sim_32     0.048701
tensor_sim_64     0.054408
tensor_sim_128    0.082004
tensor_sim_256         NaN
Name: sim, dtype: float64


|weight |rank |sim |
|--|--|--|
|log(freq)|16|0.107064|
|pmi|8|0.107581|
|iact info|16|0.108490|
|salience|16|0.125190|

# [Transitive sentence similarity](http://www.cs.ox.ac.uk/activities/compdistmeaning/GS2011data.txt) (Grefenstette and Sadrzadeh, EMNLP 2011)

## The dataset

In [13]:
svo_sim = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/GS2011data.txt',
                      sep=' ', true_values=['HIGH'], false_values=['LOW'])

In [14]:
cols_ordered = ["subject", "verb", "landmark", "object", "input", "hilo", "participant"]

In [15]:
svo_sim[cols_ordered].head()

,subject,verb,landmark,object,input,hilo,participant
0,family,provide,supply,home,4,True,participant20
1,government,provide,leave,cash,3,False,participant20
2,government,provide,supply,cash,7,True,participant20
3,man,provide,leave,money,6,True,participant20
4,man,provide,supply,money,7,True,participant20


In [17]:
svo_sim.groupby('landmark').size().sort_values(ascending=False)

landmark
visit       130
publish     130
judge       130
move        130
test        130
operate     130
satisfy     130
spell       130
purchase    125
leave       125
supply      125
bribe       125
receive     120
picture     120
state       120
express     120
depict      120
bear        120
attract     120
allege      120
dtype: int64

In [16]:
svo_sim.groupby('verb').size().sort_values(ascending=False)

verb
write      260
try        260
run        260
meet       260
provide    250
buy        250
show       240
say        240
draw       240
accept     240
dtype: int64

In [18]:
svo_sim.describe(percentiles=[])

,input
count,2500.000000
mean,3.570800
std,2.193561
min,0.000000
50%,3.000000
max,7.000000


In [19]:
svo_sim.corr(method='spearman')

,input,hilo
input,1.00000,0.51689
hilo,0.51689,1.00000


## Testing the verb tensor

In [54]:
def predict_verb(weight, rank, cutoff=2, prec_at=1, log_oov=False):
    _, index = pickle.load(open(os.path.join(tensor_dir, 'sparstensr_{}_{}.pkl').format(weight, cutoff), 
                                mode='rb'))
    ktensor, fit, n_iterations, exectimes = pickle.load(open(os.path.join(
        tensor_dir, 'ktensor_{}_{}_{}.pkl').format(weight, cutoff, rank), mode='rb'))
    # modes are ['nsubj', 'ROOT', 'dobj'].
    oov = defaultdict(int)
    def verb_pred(ser):
        try:
            predicted_ids = np.argsort((
                -ktensor.lmbda * ktensor.U[0][index['nsubj'][ser[0]]]) .dot(
                (ktensor.U[1] * ktensor.U[2][index['dobj'][ser[1]]]).T))
            return [index['ROOT'].inverse[i] for i in predicted_ids[:prec_at]]
        except KeyError as e:
            oov[e.args] += 1
            return []
    svo_sim['predicted_{}_{}'.format(weight, rank)] = svo_sim[['subject', 'object']].apply(verb_pred, axis=1)
    if log_oov:
        logging.debug(sorted(oov.items(), key=operator.itemgetter(1), reverse=True))
    for target in ['landmark', 'verb']:
        def is_good(ser):
            return ser[target] in ser['predicted_{}_{}'.format(weight, rank)]
        svo_sim['good_{}_{}_{}'.format(target, weight, rank)] = svo_sim.apply(is_good, axis=1)
        n_good = svo_sim['good_{}_{}_{}'.format(target, weight, rank)].sum()
        #major_baseline = 130 if target == 'landmark' else 260
        if hit > 0:
            logging.debug('{}\t{}\t{}\t{}'(target, weight, rank, n_good))

In [57]:
def for_weight_and_rank(max_exp_plus_one=9):
    for weight in [#'log_freq', 'pmi', 'iact_info', 'salience', 'iact_sali', 
            'log_dice']: # iact_sali
        logging.info(weight)
        for exp in range(1, max_exp_plus_one):
            rank = 2 ** exp
            logging.info(rank)
            predict_verb(weight, rank)

In [58]:
%time for_weight_and_rank()

INFO     [4] log_dice
INFO     [7] 2
INFO     [7] 4
INFO     [7] 8
DEBUG    [26] ('verb', 'log_dice', 8, 168)
INFO     [7] 16
DEBUG    [26] ('verb', 'log_dice', 16, 96)
INFO     [7] 32
DEBUG    [26] ('verb', 'log_dice', 32, 144)
INFO     [7] 64
DEBUG    [26] ('verb', 'log_dice', 64, 194)
INFO     [7] 128
DEBUG    [26] ('verb', 'log_dice', 128, 270)
INFO     [7] 256


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/permanent/home/makrai/project/verb-tensor/top_level/tensor/ktensor_log_dice_2_256.pkl'

With 44 k $\times$ 9 k $\times$ 39 k-s tenosor:

|assoc measure|rank|verb|
|-|-|-|
|pmi|256|318|
|salience|128|318|
|log-Dice|128|270|

In [41]:
cols_ordered = ["subject", "verb", "landmark", "object", "input", "hilo", "participant", 'predicted', 'good']

In [64]:
svo_sim.sum(numeric_only=True).sort_values(ascending=False).head()/svo_sim.shape[0]

input                     3.5708
hilo                      0.4652
good_verb_pmi_256         0.1272
good_verb_salience_128    0.1272
good_verb_salience_256    0.1184
dtype: float64

|verb|`* 1`|`* lmbda`|
|----|--|------|
|unnorm|130|**272**|
|norm|0|24|

Majoroty baseline...

In [74]:
target = 'verb'
svo_sim.groupby(target).size().sort_values()/svo_sim.shape[0]

verb
accept     0.096
draw       0.096
say        0.096
show       0.096
buy        0.100
provide    0.100
meet       0.104
run        0.104
try        0.104
write      0.104
dtype: float64

# [Verb prediction (tensor)](https://everest.hds.utc.fr/doku.php?id=en:lfmnips12) (Jenatton+ NIPS 2012)

In [45]:
dataset_dir = '/mnt/permanent/Language/English/Data/verb-similarity/SVO-tensor-dataset/'
def get_index(pos):
    with open(os.path.join(dataset_dir, 'svo-{}s.lst'.format(pos))) as infile:
        return {i+1: ' '.join(line.strip().split('_')[2:-1]) for i, line in enumerate(infile)}

In [46]:
svo_df = pd.read_csv(os.path.join(dataset_dir, 'svo_data_train_1000000.dat'), sep='\t', header=None, 
                     names=['subject', 'verb', 'object'])

In [47]:
index_verb = get_index('verb')
index_noun = get_index('noun')

In [48]:
svo_df.subject = svo_df.subject.apply(index_noun.get)
svo_df.verb = svo_df.verb.apply(index_verb.get)
svo_df.object = svo_df.object.apply(index_noun.get)

In [49]:
svo_df.head(10)

,subject,verb,object
0,man,swipe,credit card
1,george westinghouse,illuminate,exposition
2,personality,vanish,moment
3,fable,highlight,role
4,secretion,call,surfactant
5,device,store,multimedia
6,diversity,base,engineering
7,troop,destroy,weather station
8,flute,include,vein
9,complication,relate,emphysema
